In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import pdb
import pickle
import sys
import time
from pathlib import Path
from collections import OrderedDict as odict

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg

paths = [
    Path(".").absolute().parent,
    Path(".").absolute(),
    Path("~/Dropbox/stanford/sensitivity_analysis/goal_oriented_driving").expanduser(),
]
for path in paths:
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

from jfi import init
jaxm = init(dtype=float, device="cpu")

import pmpc
from goal_oriented_driving.dynamics import f_np as f_fn
from goal_oriented_driving.dynamics import fu_np as fu_fn
from goal_oriented_driving.dynamics import fx_np as fx_fn

In [3]:
def f_fx_fu_fn(X_prev, U_prev):
    x, u, p = X_prev, U_prev, np.array([0.3, 1.0, 1.0])
    return f_fn(x, u, p), fx_fn(x, u, p), fu_fn(x, u, p)

def get_data(solver):
    keys = ["P", "q", "A", "b", "G", "l", "u", "z_sol"]
    ret = odict([(k, jaxm.array(getattr(solver, k))) for k in keys])
    ret["z"] = ret["z_sol"]
    del ret["z_sol"]
    return ret


In [9]:
M, N, xdim, udim = 1, 30, 4, 2

Q = np.tile(np.eye(xdim), (N, 1, 1))
R = np.tile(1e0 * np.eye(udim), (N, 1, 1))
x0 = np.tile(np.ones(xdim), (1,))
X_ref, U_ref = np.zeros((N, xdim)), np.zeros((N, udim))
X_prev, U_prev = np.zeros((N, xdim)), np.zeros((N, udim))
u_limit = 0.2
u_l, u_u = -u_limit * np.ones((N, udim)), u_limit * np.ones((N, udim))

opts = dict(verbose=True, u_l=u_l, u_u=u_u)
args = (f_fx_fu_fn, Q, R, x0, X_ref, U_ref, X_prev, U_prev)

opts["reg_x"], opts["reg_u"] = 1e2, 1e1
opts["solver_settings"] = dict(
    return_solver_id=True,
    return_solver=True,
    solver_name="ecos",
    smooth_cstr="logbarrier",
    smooth_alpha=1e-1,
)
#ret = pmpc.tune_scp(*args, **opts)
opts["reg_x"], opts["reg_u"] = 1e-1, 1e-2
X, U, data = pmpc.solve(*args, max_it=100, **opts)


+------+------------+------------+------------+------------+------------+
|  it  |   elaps    |    obj     |   resid    |   reg_x    |   reg_u    |
+------+------------+------------+------------+------------+------------+
| 0001 |  2.206e-02 |  6.402e+01 |  9.618e+00 |  1.000e-01 |  1.000e-02 |
| 0002 |  4.294e-02 |  7.277e+01 |  6.127e+00 |  1.000e-01 |  1.000e-02 |
| 0003 |  6.383e-02 |  6.855e+01 |  1.166e+00 |  1.000e-01 |  1.000e-02 |
| 0004 |  8.393e-02 |  6.602e+01 |  1.605e+00 |  1.000e-01 |  1.000e-02 |
| 0005 |  1.046e-01 |  6.296e+01 |  1.225e+00 |  1.000e-01 |  1.000e-02 |
| 0006 |  1.252e-01 |  6.283e+01 |  1.625e+00 |  1.000e-01 |  1.000e-02 |
| 0007 |  1.457e-01 |  5.978e+01 |  1.131e+00 |  1.000e-01 |  1.000e-02 |
| 0008 |  1.661e-01 |  6.052e+01 |  1.569e+00 |  1.000e-01 |  1.000e-02 |
| 0009 |  1.872e-01 |  5.735e+01 |  1.011e+00 |  1.000e-01 |  1.000e-02 |
| 0010 |  2.095e-01 |  5.911e+01 |  1.709e+00 |  1.000e-01 |  1.000e-02 |
| 0011 |  2.310e-01 |  5.524e+01 |  9.

In [16]:
sol = get_data(data["solver_data"][-1]["solver"])

In [18]:
sol["G"] @ sol["z"] - sol["l"]

DeviceArray([0.02021755, 0.35371858, 0.02166781, 0.36982692, 0.02361003,
             0.37638903, 0.02618975, 0.37965846, 0.02961001, 0.3813655 ,
             0.03416613, 0.38216724, 0.04029671, 0.3823593 , 0.04866303,
             0.38208729, 0.06026625, 0.381425  , 0.07658114, 0.38040736,
             0.09951901, 0.379044  , 0.13046169, 0.37732108, 0.16714837,
             0.37519417, 0.2017138 , 0.37257703, 0.22731072, 0.36933519,
             0.24327468, 0.3652852 , 0.25176718, 0.36019207, 0.25484789,
             0.35376358, 0.25395033, 0.34564832, 0.25008179, 0.33545014,
             0.2440627 , 0.32278047, 0.23667913, 0.30738121, 0.22873229,
             0.28934921, 0.22099427, 0.26942588, 0.21411108, 0.2491384 ,
             0.20851625, 0.23047603, 0.20439938, 0.2151783 , 0.20173102,
             0.20425254, 0.20031714, 0.19805516, 0.19985778, 0.19663713],            dtype=float64)

In [ ]:
#X, U = X[0], U[0]

#ret = pmpc.tune_scp(*args, solve_fn=pmpc.accelerated_scp_solve, **opts)
#opts["rho_res_x"], opts["rho_res_u"] = ret
#X, U, data = pmpc.accelerated_scp_solve(*args, max_iters=100, **opts)
#X, U = X[0], U[0]

plt.figure()
for r in range(xdim):
    plt.plot(X[:, r], label="$x_%d$" % (r + 1))
plt.legend()
plt.tight_layout()

plt.figure()
for r in range(udim):
    plt.plot(U[:, r], label="$u_%d$" % (r + 1))
plt.legend()
plt.tight_layout()

plt.draw_all()
plt.pause(1e-1)
pdb.set_trace()